In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


66 documents
15 classes ['Branch Change', 'Course details', 'Course review', 'Double major', 'Dual Degree', 'Internship', 'Official Websites', 'Placement', 'Professor Grading', 'Projects', 'Template', 'goodbye', 'greeting', 'options', 'thanks']
138 unique lemmatized words ["'s", ',', '.', '36', 'a', 'about', 'all', 'any', 'anyone', 'anytime', 'are', 'average', 'awesome', 'be', 'branch', 'bye', 'came', 'can', 'ce', 'ce262a', 'change', 'chatting', 'civil', 'company', 'consider', 'core', 'could', 'course', 'cpi', 'credit', 'criterion', 'ctc', 'day', 'degree', 'department', 'detail', 'difficult', 'do', 'done', 'double', 'dual', 'engineering', 'eso202a', 'eso208a', 'for', 'from', 'get', 'getting', 'give', 'go', 'good', 'goodbye', 'grabe', 'grade', 'grading', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'if', 'iitk', 'important', 'in', 'industrial', 'intern', 'internship', 'internâ', 'into', 'inâ', 'is', 'issue', 'it', 'later', 'link', 'major', 'manage', 'm

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_4936/2859036294.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
C:\Users\LENOVO\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
14/14 [==============================] - 1s 3ms/step - loss: 2.7119 - accuracy: 0.0606
Epoch 2/200
14/14 [==============================] - 0s 1ms/step - loss: 2.6362 - accuracy: 0.0909
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 2.5088 - accuracy: 0.1970
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 2.3961 - accuracy: 0.2121
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 2.3578 - accuracy: 0.2273
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 2.3144 - accuracy: 0.3030
Epoch 7/200
14/14 [==============================] - 0s 3ms/step - loss: 2.0962 - accuracy: 0.4242
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 1.9974 - accuracy: 0.3788
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 1.9319 - accuracy: 0.4242
Epoch 10/200
14/14 [==============================] - 0s 3ms/step - loss: 1.8087 - accuracy: 0.3788
Epoch 11/